In [1]:
import logging
import os
import sys

# Set up logging configuration
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(sys.stdout),
        logging.FileHandler('orchestrator.log')
    ]
)

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))


from autogen.agentchat.contrib.orchestrator_agent import OrchestratorAgent
from autogen.agentchat import AssistantAgent, UserProxyAgent
import json

config_list = [
    {
        "model": os.environ.get("MODEL"),
        "api_key": os.environ.get("API_KEY"),
        "base_url": os.environ.get("BASE_URL", None)
    }
]

llm_config = {
    "config_list": config_list,
    "temperature": 0,
    "timeout": 120
}


coder = AssistantAgent(
    name="coder",
    llm_config=llm_config,
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    system_message="You are a helpful AI coding assistant. You write clean, efficient code and explain your solutions clearly.  End your massage with TERMINATE when everything is done.",
    max_consecutive_auto_reply=10,
    human_input_mode="NEVER",
    description="coder. Writes and executes code to solve tasks.",
)


orchestrator = OrchestratorAgent(
    name="Orchestrator",
    llm_config=llm_config,
    agents=[coder],
    max_consecutive_auto_reply=10,
    max_stalls_before_replan=2,  #3
    max_replans=3, #3 
    return_final_answer=True,
    description="An orchestrator that manages conversation flow and handles errors."
)

task = """write and execute a python prints the prime numbers between 1 and 1000"""
orchestrator.initiate_chat(message=task)
        



2024-11-20 19:46:40,859 - flaml - WARNING - flaml.automl is not available. Please install flaml[automl] to enable AutoML functionalities.
2024-11-20 19:46:40,996 - autogen.agentchat.contrib.orchestrator_agent - INFO - Initializing OrchestratorAgent...
2024-11-20 19:46:40,997 - autogen.agentchat.contrib.orchestrator_agent - INFO - Starting orchestrated conversation with task: write and execute a python prints the prime numbers between 1 and 1000

>>>>>>>> USING AUTO REPLY...
2024-11-20 19:46:41,015 - autogen.agentchat.contrib.orchestrator_agent - INFO - Task analysis complete: 1. **GIVEN OR VERIFIED FACTS:**

   - The task is to write and execute a Python program.
   - The program should print prime numbers.
   - The range of numbers is 1 to 1000 (inclusive).


2. **FACTS TO LOOK UP:**

   - Definition of a prime number. (Source: Mathematical textbooks, online encyclopedias like Wikipedia)
   - Efficient algorithms for finding prime numbers within a given range. (Source:  Computational 

'Okay, we\'ve successfully completed your request to write and execute a Python program that prints prime numbers between 1 and 1000.\n\nHere\'s the code:\n\n```python\ndef is_prime(num):\n  """Checks if a number is prime."""\n  if num <= 1:\n    return False\n  if num <= 3:\n    return True\n  if num % 2 == 0 or num % 3 == 0:\n    return False\n  i = 5\n  while i * i <= num:\n    if num % i == 0 or num % (i + 2) == 0:\n      return False\n    i += 6\n  return True\n\nprint("Prime numbers between 1 and 1000:")\nfor num in range(2, 1001):  #Start from 2 as 1 is not prime\n  if is_prime(num):\n    print(num, end=" ")\n\n```\n\nThis code defines a function `is_prime` which efficiently checks for primality using a optimized algorithm.  The main part of the code then iterates through numbers 2 to 1000, calling `is_prime` for each and printing those that are prime.\n\n\n**Key Findings/Outcomes:**\n\n* The program successfully identifies and prints all prime numbers between 1 and 1000.\n* The